In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from datetime import datetime
import matplotlib.pyplot as plt
from skimage import morphology
from skimage.measure import label, regionprops
import pandas as pd
import matplotlib.patches as patches
from skimage import measure
from skimage.color import rgb2gray
from skimage.morphology import disk
import re
import datetime
from skimage.io import imsave

**Set Directory Paths**

!!!!!!!!!!!!!!!!!!!
SET path_clustered_data to the clustering result you want. This expecting our custom clustering as it is indexed to work only with that one

If you want to use this to work with another clustering technique you will need to how the data is imported

In [ ]:
#drive path to directory containg datasets
# need to copy the path of the directory you just clustered.
path_clustered_dataset = "..\Data Sets\Clustered Datasets\Custom_20230518_013052"

# the raw dataset as the bounding box will be drawn over this image
path_raw_dataset= "../Data Sets/Raw Datasets"

#drive path to output directory for drawn and bounded images
path_output = "../Data Sets/Output Images - bounded protein particaless"

**Load datasets (clustered and raw)**

**Load Clustered Images**

Loads in the clustered images from path_clustered_images

Be sure to update this inorder to access the most recent cluster results

This is designed for use with the custom clustering method, and will not work with the other clustering methods

This is due to the fact that FCM and K-means do not cluster similarly to a single cluster (best cluster in FMC for one image will differ for the next) this prevents us from creating a way to quickly process large datasets as you would have to go in by hand and select the best cluster

custom clustering will always put the best cluster as cluster 2

So this import will only import cluster 2

In [ ]:
clustered_dataset = {}
# Loop through all folders in the output directory and import images
for folder in sorted(os.listdir(path_clustered_dataset)):
    folder_path = os.path.join(path_clustered_dataset, folder)
    data = []
    
    # Get a list of all file names that end with 'cluster2.png'
    cluster2_files = [f for f in os.listdir(folder_path) if f.endswith('cluster2.png')]
    
    # Define a custom sort key that sorts based on the numeric part of the filename
    def sort_key(file_name):
        # Extract the number from the filename (assuming the filename format is 'imageX_cluster2.png')
        number = int(file_name.split('_')[0][5:])
        return number
    
    # Sort the filenames using the custom sort key
    cluster2_files.sort(key=sort_key)
    
    for file_name in cluster2_files:
        file_path = os.path.join(folder_path, file_name)
        image = cv2.imread(file_path)  # read image as grayscale
        if image is not None:
            # Convert the image to grayscale
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            data.append(gray_image)

    # Convert data to numpy arrays
    data = np.array(data)
    # Add the data to the Clustered_dataset dictionary
    clustered_dataset[folder] = data

**Import Raw Data Set**

imports the raw data so that we can draw the bounding box to identify the protien particles.

We define an image resize function so that it will have the same shape as the clustered images (otherwise the bounding boxes will not be alligned)

In [ ]:
def resize_image(image, target_size):
    height, width = image.shape[:2]
    target_height, target_width = target_size

    # Calculate the aspect ratio and the new dimensions
    aspect_ratio = width / height
    new_width = target_width
    new_height = int(target_width / aspect_ratio)

    if new_height > target_height:
        new_height = target_height
        new_width = int(target_height * aspect_ratio)

    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)

    # Add padding to the resized image to match the target size
    pad_height = target_height - new_height
    pad_width = target_width - new_width
    padding = [(pad_height // 2, pad_height - pad_height // 2), 
               (pad_width // 2, pad_width - pad_width // 2), 
               (0, 0)]  # No padding for the color channel

    padded_image = np.pad(resized_image, padding, mode='constant', constant_values=0)

    return padded_image

In [ ]:
# Empty dictionary to store the our raw image data
raw_dataset = {}
resized_raw_dataset = {}

target_size = (256, 256)  # Set your target size here

# Loop through all folders in the output directory and import images
for folder in sorted(os.listdir(path_raw_dataset)):
    folder_path = os.path.join(path_raw_dataset, folder)
    data = []
    
    # Get a sorted list of all file names
    file_names = sorted(os.listdir(folder_path))
    
    for file_name in file_names:
        # Import data
        file_path = os.path.join(folder_path, file_name)
        image = cv2.imread(file_path)
        if image is not None:
            # Resize the image using the new function
            resized_image = resize_image(image, target_size)
            data.append(resized_image)

    # Convert data to numpy arrays
    data = np.array(data)
    # Add the data to the raw_dataset dictionary
    raw_dataset[folder] = data

    # Store the resized images in the resized_dataset dictionary
    resized_raw_dataset[folder] = data

Sample of raw vs clustered data

Clustered image is grayscale but is shown as RGB, this is why the color is off, this has no effect on our program and only is an issue here (Though not really an issue)

In [ ]:
# Create a new figure with 2 subplots (1 row, 2 columns)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Get the images
index = 1
image1 = resized_raw_dataset['Apoferritin'][index]  # Assuming you want to display the first image for the specified label
image2 = clustered_dataset['Apoferritin'][index]  # Assuming you want to display the first image for the specified label

# Display the images
axes[0].imshow(image1)
axes[0].axis('off')  # Turn off axis labels
axes[0].set_title('Resized Raw Image')

axes[1].imshow(image2)
axes[1].axis('off')  # Turn off axis labels
axes[1].set_title('Clustered Image')

# Display the figure
plt.show()

**Binary image processing**

This stage is really dependent on the clusterd data.

In this project we used 4 diffrent data sets of varying qulity. To get the best results from one would pretty much ruin the others.

right now I have it set up so that they are all okay, but each would preform better on its own by adjusting mophological opening and remove small objects

experiment with the image cleaning and fine tunning it to work with the specific datasets used more so

since the specificastions of each dataset is diffrent, we apply our binary masking operations tailored for each data set i.e. images in apofferitain are round, while in Beta and ribosoms they are not round at all.

so we must apply our mask based on the data sets

In [ ]:
# Empty dictionary to store the cleaned binary images
cleaned_dataset = {}

# Define different structuring elements for different datasets
selem_dict = {
    "Apoferritin": None,
    "Beta-galactosidase": None,
    "KLH": morphology.disk(2),
    "Ribosome": None,
}
# Define different structuring elements for different datasets
selem_dict2 = {
    "Apoferritin": None,
    "Beta-galactosidase": None,
    "KLH": None,
    "Ribosome": None,
}
# Define different area thresholds for different datasets
area_threshold_dict = {
    "Apoferritin": 150,
    "Beta-galactosidase": 10,
    "KLH": 250,
    "Ribosome": 10,
}

# Define different min_size for different datasets
min_size_dict = {
    "Apoferritin": 100,
    "Beta-galactosidase": 25,
    "KLH": 50,
    "Ribosome": 25,
}

for folder, images in clustered_dataset.items():
    cleaned_images = []
    for img in images:
        # Convert grayscale image to binary image
        img_binary = img > 0
        # Perform morphological closing with the dataset-specific structuring element
        closed = morphology.closing(img_binary, selem_dict[folder])
        
        # Fill holes in the binary image with the dataset-specific area threshold
        filled = morphology.remove_small_holes(closed, area_threshold=area_threshold_dict[folder], connectivity=1)
        # Remove small objects with the dataset-specific min_size
        cleaned = morphology.remove_small_objects(filled, min_size=min_size_dict[folder], connectivity=1)
        # # Perform morphological opening with a larger structuring element
        # opened = morphology.opening(cleaned, selem_dict2[folder])
        # # Remove small objects with the dataset-specific min_size
        # cleaned = morphology.remove_small_objects(opened, min_size=min_size_dict[folder], connectivity=1)


        # # Label connected components in the image
        # labels = label(cleaned)
        # # Measure properties of labeled regions
        # properties = regionprops(labels)
        # # Create a mask of objects that are sufficiently round
        # roundness_threshold = .9  # adjust this value based on your needs
        # roundness_mask = np.zeros_like(opened, dtype=bool)
        # for prop in properties:
        #     if prop.eccentricity < roundness_threshold:  # lower eccentricity means more round
        #         roundness_mask = np.logical_or(roundness_mask, labels == prop.label)

        # # Apply the roundness mask to the image
        # round_objects = opened & roundness_mask

        cleaned_images.append(cleaned)

    cleaned_dataset[folder] = cleaned_images

display binary image processing results


In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))  # Adjust figsize as needed

image1_orig = clustered_dataset['Apoferritin'][1]
image2_orig = clustered_dataset['Beta-galactosidase'][1]
image3_orig = clustered_dataset['KLH'][1]
image4_orig = clustered_dataset['Ribosome'][1]

image1_cleaned = cleaned_dataset['Apoferritin'][1].astype(np.float32)
image2_cleaned = cleaned_dataset['Beta-galactosidase'][1].astype(np.float32)
image3_cleaned = cleaned_dataset['KLH'][1].astype(np.float32)
image4_cleaned = cleaned_dataset['Ribosome'][1].astype(np.float32)

# Plot original images
ax[0, 0].imshow(image1_orig, cmap='gray')
ax[0, 0].axis('off')
ax[0, 0].set_title('Apoferritin (Original)')

ax[0, 1].imshow(image2_orig, cmap='gray')
ax[0, 1].axis('off')
ax[0, 1].set_title('Beta-galactosidase (Original)')

ax[0, 2].imshow(image3_orig, cmap='gray')
ax[0, 2].axis('off')
ax[0, 2].set_title('KLH (Original)')

ax[0, 3].imshow(image4_orig, cmap='gray')
ax[0, 3].axis('off')
ax[0, 3].set_title('Ribosome (Original)')

# Plot cleaned images
ax[1, 0].imshow(image1_cleaned, cmap='gray')
ax[1, 0].axis('off')
ax[1, 0].set_title('Apoferritin (Cleaned)')

ax[1, 1].imshow(image2_cleaned, cmap='gray')
ax[1, 1].axis('off')
ax[1, 1].set_title('Beta-galactosidase (Cleaned)')

ax[1, 2].imshow(image3_cleaned, cmap='gray')
ax[1, 2].axis('off')
ax[1, 2].set_title('KLH (Cleaned)')

ax[1, 3].imshow(image4_cleaned, cmap='gray')
ax[1, 3].axis('off')
ax[1, 3].set_title('Ribosome (Cleaned)')

plt.show()

**Region properties**

This is where a lot of improvemnts can be made, the current iteration of region defining is a very simple version

possible explorative experimentations:


*   different threshold values 
*   watershed segmentation
*   active contour models
*   level set methods










In [ ]:
# Empty dictionary to store the region properties
region_props = {}

for folder, images in cleaned_dataset.items():
    folder_props = []
    for img in images:
        # Label the connected components in the image
        labeled_img = label(img, connectivity=img.ndim)

        # Calculate properties of the labeled regions
        props = regionprops(labeled_img)

        folder_props.append(props)
    
    region_props[folder] = folder_props

**Filter the Regions**

filter these by size

Can experiment with the size filters and other possible parameters to try and get better bounding boxes

In [ ]:
# Empty dictionary to store the region properties and roundness
region_props = {}

# Define your roundness_threshold here
roundness_threshold = 0.7  #adjust as needed

for folder, images in cleaned_dataset.items():
    folder_props_and_roundness = []  # This will store both properties and roundness for each image
    for img in images:
        # Label the connected components in the image
        labeled_img = label(img, connectivity=img.ndim)

        # Calculate properties of the labeled regions
        props = regionprops(labeled_img)

        # Calculate roundness for each region and store in a list
        roundness = [4 * np.pi * prop.area / (prop.perimeter ** 2) if prop.perimeter != 0 else 0 for prop in props]

        # Create a dictionary for each image storing both properties and roundness
        prop_and_roundness = {"props": props, "roundness": roundness}

        # Append this dictionary to the folder_props_and_roundness list
        folder_props_and_roundness.append(prop_and_roundness)

    # Add to the main dictionary
    region_props[folder] = folder_props_and_roundness

In [ ]:
# Filter out large and small regions
# look into filtering by percentaile ie(drop the smallest 10% and top 10$ of identifyied regions)
min_area = 100  # Adjust as needed
max_area = 500  # Adjust as needed

for folder, props_with_roundness in region_props.items():
    filtered_props_with_roundness = []
    for prop_dict in props_with_roundness:
        props = prop_dict["props"]
        roundness = prop_dict["roundness"]

        # Debugging print statement
        print(f"Initial number of regions: {len(props)}")

        # Filter properties and roundness in synchronization
        filtered_props = []
        filtered_roundness = []
        for prop, round_val in zip(props, roundness):
            if min_area <= prop.area <= max_area:
                filtered_props.append(prop)
                filtered_roundness.append(round_val)
        
        # Debugging print statement
        print(f"Number of regions after filtering: {len(filtered_props)}")

        # Store the filtered properties and roundness in a dictionary
        filtered_dict = {"props": filtered_props, "roundness": filtered_roundness}
        filtered_props_with_roundness.append(filtered_dict)

    region_props[folder] = filtered_props_with_roundness

**Rectangle Drawing - Raw and cleaned images images**

draw our regions on both the clustered and raw images

In [ ]:
def draw_regions_on_image(image, regions):
    # Ensure the image data is in the correct format
    image = np.array(image, dtype=np.uint8)
    
    for region_list in regions:
        for region in region_list:
            minr, minc, maxr, maxc = region.bbox
            cv2.rectangle(image, (minc, minr), (maxc, maxr), (0, 255, 0), 1)  # Reduced rectangle thickness

            # Draw the centroid
            y, x = region.centroid
            cv2.drawMarker(image, (int(x), int(y)), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1, line_type=cv2.LINE_8)  # Reduced marker size and thickness
    return image


In [ ]:
def draw_regions_on_dataset(dataset, region_props):
    dataset_with_regions = {}
    
    for folder, images in dataset.items():
        folder_images = []
        for image in images:
            # Get the region props for this image
            prop_dicts = region_props.get(folder, None)
            if prop_dicts is None:
                print(f"No regions found for folder: {folder}")
                continue

            # We now pass a list of RegionProperties objects to the draw_regions_on_image function
            props = [prop_dict['props'] for prop_dict in prop_dicts]
            img_with_regions = draw_regions_on_image(image, props)
            
            if img_with_regions is None:
                print(f"Error drawing regions on image in folder: {folder}")
                continue
                
            folder_images.append(img_with_regions)
            
        dataset_with_regions[folder] = folder_images

    return dataset_with_regions if dataset_with_regions else None

In [ ]:
# Draw the bounding boxes
cleaned_with_regions = draw_regions_on_dataset(cleaned_dataset, region_props)
raw_with_regions = draw_regions_on_dataset(resized_raw_dataset, region_props)

**Save our output images**

In [ ]:
#check if directory is there, if not create it
# Check if the directory exists
if not os.path.isdir(path_output):
    # Create the directory
    os.makedirs(path_output)

In [ ]:
# Get the current timestamp
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create a new directory for the marked images
marked_images_dir = os.path.join(path_output, f'{timestamp}_MarkedImages')
os.makedirs(marked_images_dir, exist_ok=True)

for folder, images in raw_with_regions.items():
    # Create a new directory for this label
    label_dir = os.path.join(marked_images_dir, folder)
    os.makedirs(label_dir, exist_ok=True)
    
    for i, image in enumerate(images):
        # Save the image
        imsave(os.path.join(label_dir, f'image{i+1:03d}.png'), image)

Show Sample outputs

In [ ]:
def display_images_side_by_side(unbounded_images, raw_images, cleaned_images):
    fig, axs = plt.subplots(len(unbounded_images), 3, figsize=(15, len(unbounded_images)*5))

    for i, (unbounded_image, raw_image, cleaned_image) in enumerate(zip(unbounded_images, raw_images, cleaned_images)):
        if unbounded_image.ndim == 3:  # If the image is RGB, no need to use a colormap
            axs[i, 0].imshow(unbounded_image)
        else:  # If the image is grayscale, use the 'gray' colormap
            axs[i, 0].imshow(unbounded_image, cmap='gray')
        axs[i, 0].set_title('Unbounded Raw Image')
        axs[i, 0].axis('off')

        if raw_image.ndim == 3:  # If the image is RGB, no need to use a colormap
            axs[i, 1].imshow(raw_image)
        else:  # If the image is grayscale, use the 'gray' colormap
            axs[i, 1].imshow(raw_image, cmap='gray')
        axs[i, 1].set_title('Bounded Raw Image')
        axs[i, 1].axis('off')

        if cleaned_image.ndim == 3:  # If the image is RGB, no need to use a colormap
            axs[i, 2].imshow(cleaned_image)
        else:  # If the image is grayscale, use the 'gray' colormap
            axs[i, 2].imshow(cleaned_image, cmap='gray')
        axs[i, 2].set_title('Bounded Clustered Image')
        axs[i, 2].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
unbounded_images = [resized_raw_dataset['Apoferritin'][1], resized_raw_dataset['Beta-galactosidase'][1], resized_raw_dataset['KLH'][1], resized_raw_dataset['Ribosome'][1]]
cleaned_images = [cleaned_with_regions['Apoferritin'][1], cleaned_with_regions['Beta-galactosidase'][1], cleaned_with_regions['KLH'][1], cleaned_with_regions['Ribosome'][1]]
raw_images = [raw_with_regions['Apoferritin'][1], raw_with_regions['Beta-galactosidase'][1], raw_with_regions['KLH'][1], raw_with_regions['Ribosome'][1]]

display_images_side_by_side(unbounded_images, raw_images, cleaned_images)